# Springboard Capstone 02 - Energy price prediction based on weather
## Pre-processing

In [22]:
#Import libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil import tz
from pytz import timezone
from pandas import Grouper
from pandas import concat
from pandas import DataFrame
import matplotlib.ticker as mtick

In [23]:
#import dataframe and deal with date column
eng_wea_df = pd.read_csv('data/energy_weather.csv')
eng_wea_df['date'] = pd.to_datetime(eng_wea_df['date'], utc=False)
#Convert to Madrid timezone
#eng_wea_df.date = eng_wea_df.date.dt.tz_localize('Europe/Madrid')
#eng_wea_df.date = eng_wea_df.date.date()
eng_wea_df['date'] = pd.to_datetime(eng_wea_df['date']).dt.date

#eng_wea_df.set_index('date', inplace=True)
eng_wea_df.drop(columns='Unnamed: 0', inplace=True)

eng_wea_df.head(15)

,date,city_name,temp_daily_max,pressure_daily_max,humidity_daily_max,wind_speed_daily_max,rain_1h_daily_max,rain_3h_daily_max,snow_3h_daily_max,clouds_all_daily_max,...,generation wind onshore_daily_max,forecast solar day ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual,gen_total,month,day_of_week
0,2015-01-01,Barcelona,284.602312,1038.0,100.0,7.0,0.0,0.0,0.0,0.0,...,6378.0,3842.0,6436.0,30739.0,29571.0,62.69,75.64,28973.0,1,Thursday
1,2015-01-01,Bilbao,284.591687,1036.0,97.0,1.0,0.0,0.0,0.0,24.0,...,6378.0,3842.0,6436.0,30739.0,29571.0,62.69,75.64,28973.0,1,Thursday
2,2015-01-01,Madrid,283.308312,1036.0,75.0,1.0,0.0,0.0,0.0,0.0,...,6378.0,3842.0,6436.0,30739.0,29571.0,62.69,75.64,28973.0,1,Thursday
3,2015-01-01,Seville,286.994000,1043.0,84.0,4.0,0.0,0.0,0.0,0.0,...,6378.0,3842.0,6436.0,30739.0,29571.0,62.69,75.64,28973.0,1,Thursday
4,2015-01-01,Valencia,285.277344,1035.0,84.0,2.0,0.0,0.0,0.0,0.0,...,6378.0,3842.0,6436.0,30739.0,29571.0,62.69,75.64,28973.0,1,Thursday
5,2015-01-02,Barcelona,286.328688,1039.0,100.0,2.0,0.0,0.0,0.0,64.0,...,13232.0,4120.0,13329.0,33968.0,33668.0,52.01,83.33,33578.0,1,Friday
6,2015-01-02,Bilbao,284.845344,1038.0,95.0,2.0,0.0,0.0,0.0,22.0,...,13232.0,4120.0,13329.0,33968.0,33668.0,52.01,83.33,33578.0,1,Friday
7,2015-01-02,Madrid,284.945344,1038.0,79.0,1.0,0.0,0.0,0.0,32.0,...,13232.0,4120.0,13329.0,33968.0,33668.0,52.01,83.33,33578.0,1,Friday
8,2015-01-02,Seville,286.864312,1045.0,89.0,4.0,0.0,0.0,0.0,0.0,...,13232.0,4120.0,13329.0,33968.0,33668.0,52.01,83.33,33578.0,1,Friday
9,2015-01-02,Valencia,287.553688,1037.0,82.0,2.0,0.0,0.0,0.0,0.0,...,13232.0,4120.0,13329.0,33968.0,33668.0,52.01,83.33,33578.0,1,Friday


In [24]:
eng_wea_df.columns = eng_wea_df.columns.get_level_values(0)
eng_wea_df.head(15)

,date,city_name,temp_daily_max,pressure_daily_max,humidity_daily_max,wind_speed_daily_max,rain_1h_daily_max,rain_3h_daily_max,snow_3h_daily_max,clouds_all_daily_max,...,generation wind onshore_daily_max,forecast solar day ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual,gen_total,month,day_of_week
0,2015-01-01,Barcelona,284.602312,1038.0,100.0,7.0,0.0,0.0,0.0,0.0,...,6378.0,3842.0,6436.0,30739.0,29571.0,62.69,75.64,28973.0,1,Thursday
1,2015-01-01,Bilbao,284.591687,1036.0,97.0,1.0,0.0,0.0,0.0,24.0,...,6378.0,3842.0,6436.0,30739.0,29571.0,62.69,75.64,28973.0,1,Thursday
2,2015-01-01,Madrid,283.308312,1036.0,75.0,1.0,0.0,0.0,0.0,0.0,...,6378.0,3842.0,6436.0,30739.0,29571.0,62.69,75.64,28973.0,1,Thursday
3,2015-01-01,Seville,286.994000,1043.0,84.0,4.0,0.0,0.0,0.0,0.0,...,6378.0,3842.0,6436.0,30739.0,29571.0,62.69,75.64,28973.0,1,Thursday
4,2015-01-01,Valencia,285.277344,1035.0,84.0,2.0,0.0,0.0,0.0,0.0,...,6378.0,3842.0,6436.0,30739.0,29571.0,62.69,75.64,28973.0,1,Thursday
5,2015-01-02,Barcelona,286.328688,1039.0,100.0,2.0,0.0,0.0,0.0,64.0,...,13232.0,4120.0,13329.0,33968.0,33668.0,52.01,83.33,33578.0,1,Friday
6,2015-01-02,Bilbao,284.845344,1038.0,95.0,2.0,0.0,0.0,0.0,22.0,...,13232.0,4120.0,13329.0,33968.0,33668.0,52.01,83.33,33578.0,1,Friday
7,2015-01-02,Madrid,284.945344,1038.0,79.0,1.0,0.0,0.0,0.0,32.0,...,13232.0,4120.0,13329.0,33968.0,33668.0,52.01,83.33,33578.0,1,Friday
8,2015-01-02,Seville,286.864312,1045.0,89.0,4.0,0.0,0.0,0.0,0.0,...,13232.0,4120.0,13329.0,33968.0,33668.0,52.01,83.33,33578.0,1,Friday
9,2015-01-02,Valencia,287.553688,1037.0,82.0,2.0,0.0,0.0,0.0,0.0,...,13232.0,4120.0,13329.0,33968.0,33668.0,52.01,83.33,33578.0,1,Friday


In [25]:
eng_wea_df.set_index('date', inplace=True)

In [26]:
eng_wea_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7305 entries, 2015-01-01 to 2018-12-31
Data columns (total 33 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   city_name                                              7305 non-null   object 
 1   temp_daily_max                                         7305 non-null   float64
 2   pressure_daily_max                                     7305 non-null   float64
 3   humidity_daily_max                                     7305 non-null   float64
 4   wind_speed_daily_max                                   7305 non-null   float64
 5   rain_1h_daily_max                                      7305 non-null   float64
 6   rain_3h_daily_max                                      7305 non-null   float64
 7   snow_3h_daily_max                                      7305 non-null   float64
 8   clouds_all_daily_max                  

## 2.0 Converting Categorical Features

In [27]:
eng_wea_df = pd.get_dummies(eng_wea_df, columns=['city_name', 'month', 'day_of_week'])
eng_wea_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7305 entries, 2015-01-01 to 2018-12-31
Data columns (total 54 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   temp_daily_max                                         7305 non-null   float64
 1   pressure_daily_max                                     7305 non-null   float64
 2   humidity_daily_max                                     7305 non-null   float64
 3   wind_speed_daily_max                                   7305 non-null   float64
 4   rain_1h_daily_max                                      7305 non-null   float64
 5   rain_3h_daily_max                                      7305 non-null   float64
 6   snow_3h_daily_max                                      7305 non-null   float64
 7   clouds_all_daily_max                                   7305 non-null   float64
 8   generation biomass_daily_max          

In [30]:
eng_wea_df.drop(columns='generation marine_daily_max', inplace=True)

## 3.0 Splitting data

In [31]:
from sklearn.model_selection import train_test_split

X = eng_wea_df.drop(columns='price actual')
y = eng_wea_df['price actual']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

## 4.0 Scaling Data

In [32]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_tr_scaled = scaler.transform(X_train)
X_te_scaled = scaler.transform(X_test)